In [ ]:
from PyPDF2 import PdfReader

**The Document Location**

In [ ]:
pdfPath = "python_fundamentals.pdf"

**Function for loading the file**

In [ ]:
def loadFile(path):
    reader = PdfReader(path)
    allTexts = ""

    for page in reader.pages:
        text = page.extract_text()
        if text:
            allTexts += text + "\n"

    return allTexts

**Splitting data into chunks**

In [ ]:
def splitIntoChunks(text, chunkSize = 300, overlap = 50):
    chunks = []
    start = 0

    text_length = len(text)

    while start < text_length:
        end = start + chunkSize
        chunk = text[start:end]
        chunks.append(chunk)
        start = end - overlap

    return chunks

In [ ]:
text = loadFile(pdfPath)
print("total characters : ",len(text))

chunks = splitIntoChunks(text)
print("total chunks created : ",len(chunks))

**Loading Embedding Model**

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

**ChromaDB setup**

In [ ]:
import chromadb

# creating chromadb instance which will store all chunks and embeddings in it

chroma_client = chromadb.Client()
collection  = chroma_client.create_collection(name = "pdf_readers" )

**Inserting data into CHROMADB**

In [ ]:
for i , chunk in enumerate(chunks):
    emb = model.encode([chunk])[0].tolist()
    collection.add(
        documents = [chunk],
        embeddings = [emb],
        ids = [f"chunk-{i}"]
    )

print("vector db is ready")

### Integrating LLM through Ollama

In [ ]:
import ollama

def runLLM(prompt):
    response = ollama.generate(
        model = "tinyllama", # you have to pull "ollama pull tinyllama" on your local ollama terminal
        prompt = prompt
    )
    return response["response"]

**RAG Function**

In [ ]:
def rag(query):

    print("The Question is : ",query)

    # step1 : embed the query
    query_emb = model.encode([query])[0].tolist()

    # step2 : retrieve chunks
    results = collection.query(
        query_embeddings = [query_emb],# Compares it with every stored chunk embedding, Uses cosine similarity, Finds closest vectors

        n_results = 2   # top 2 similar matches
    )

    chunks = results["documents"][0]
    context = "\n\n".join(chunks) # the only knowledge we will give to llm to think for the query


    # step3 : Building the prompt
    prompt = f"""Use ONLY the context below to answer the question.CONTEXT:{context}QUESTION: {query}If answer is not in the context, say: "I don't know"."""


    # step4: run llm 
    answer = runLLM(prompt)
    print("Answer:\n")
    print(answer)

**Asking question from user and pass it to the LLM**

In [ ]:
userQuery = input("What's your question ?")
rag(userQuery)